In [1]:
import time
import statistics
import pandas as pd
from datetime import datetime
from appium import webdriver
from appium.options.android import UiAutomator2Options
from appium.webdriver.common.appiumby import AppiumBy
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [ ]:
# ===================== [설정 영역] =====================
APP_PACKAGE = "com.jobkorea.app"
APP_ACTIVITY = None  # 자동 감지
DEVICE_NAME = "Galaxy S25"
APPIUM_SERVER_URL = "http://127.0.0.1:4723"

# [계정 정보]
LOGIN_ID = "아이디 입력" # 아이디
LOGIN_PW = "비밀번호 입력" # 비밀번호

# 반복 횟수
REPEAT_COUNT = 10


# ========= 핸드폰 교체시 필수 수정 =========

# [팝업 닫기 좌표] (스크롤 하기 전에 누를 좌표)
POPUP_CLOSE_X = 957
POPUP_CLOSE_Y = 1856

# 스크롤 할 횟수
scroll_num = 4

# 로그아웃 버튼 좌표 (스크롤 후)
LOGOUT_X = 920 
LOGOUT_Y = 1911



In [3]:
# 메인 테스트
def run_jobkorea_test():
    options = UiAutomator2Options()
    options.device_name = DEVICE_NAME
    options.app_package = APP_PACKAGE
    options.app_wait_activity = "*"
    options.automation_name = "UiAutomator2"
    options.no_reset = True
    options.new_command_timeout = 300

    print(f"--- 잡코리아 로그인 성능 측정 ---")
    
    driver = webdriver.Remote(APPIUM_SERVER_URL, options=options)
    wait = WebDriverWait(driver, 20)
    
    results = []

    try:
        for i in range(1, REPEAT_COUNT + 1):
            print(f"\n🔄 [ {i} / {REPEAT_COUNT} ] 회차 수행 중...")

            # ---------------------------------------------------------
            # 1. 메인 진입 & MY 클릭
            # ---------------------------------------------------------
            print("📲 [1] MY 메뉴 진입")
            try:
                WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((AppiumBy.XPATH, "//*[@text='앗!뜨공' or contains(@content-desc, '앗!뜨공')]"))
                )
            except: pass 

            try:
                wait.until(EC.element_to_be_clickable((AppiumBy.ID, "com.jobkorea.app:id/rl_my"))).click()
                print("   ✅ MY 버튼 클릭 완료")
            except:
                print("   ❌ MY 버튼 찾기 실패")
                continue

            # ---------------------------------------------------------
            # 2. '다른 아이디로 로그인'
            # ---------------------------------------------------------
            try:
                target_xpath = '//android.widget.TextView[@resource-id="com.jobkorea.app:id/tvAnotherLogin"]'
                WebDriverWait(driver, 3).until(EC.element_to_be_clickable((AppiumBy.XPATH, target_xpath))).click()
                print("   ✅ '다른 아이디로 로그인' 클릭")
            except:
                pass

            # ---------------------------------------------------------
            # 3. 아이디 / 비밀번호 입력
            # ---------------------------------------------------------
            print("⌨️ [2] 정보 입력")
            try:
                id_field = wait.until(EC.presence_of_element_located((AppiumBy.ID, "com.jobkorea.app:id/editTextId")))
                id_field.clear()
                id_field.send_keys(LOGIN_ID)
                
                pw_field = driver.find_element(AppiumBy.ID, "com.jobkorea.app:id/editTextPassword")
                pw_field.clear()
                pw_field.send_keys(LOGIN_PW)
            except:
                print("   ❌ 입력창 찾기 실패")
                continue

            try: driver.hide_keyboard()
            except: pass

            # ---------------------------------------------------------
            # 4. 로그인 수행 (측정)
            # ---------------------------------------------------------
            print("⏱️ [3] 로그인 시작")
            login_btn = driver.find_element(AppiumBy.ID, "com.jobkorea.app:id/bt_login")
            
            start_time = time.time()
            login_btn.click()

            # 5. 완료 확인
            try:
                wait.until(EC.presence_of_element_located((AppiumBy.XPATH, "//*[@text='회원정보' or @text='이력서 관리']")))
                end_time = time.time()
                elapsed = end_time - start_time
                print(f"   🎉 로그인 성공! ({elapsed:.4f}초)")
                results.append(elapsed)
            except:
                print("   ❌ 로그인 시간 초과")
                continue

            # ---------------------------------------------------------
            # 6. 로그아웃 (팝업 좌표 닫기 -> 스크롤 10번 -> 로그아웃 좌표 클릭)
            # ---------------------------------------------------------
            print("🚪 [4] 로그아웃 진행 (좌표 클릭)")
            
            # (1) [핵심] 팝업 닫기 좌표 클릭
            print(f"   👆 팝업 닫기")
            try:
                driver.tap([(POPUP_CLOSE_X, POPUP_CLOSE_Y)])
                time.sleep(1) 
            except Exception as e:
                print(f"   ⚠️ 팝업 닫기 실패: {e}")

            # (2) 화면 끝까지 스크롤
            print(f"   📜 화면 최하단으로 스크롤 {scroll_num}회...")
            for _ in range(scroll_num): 
                size = driver.get_window_size()
                driver.swipe(size['width']*0.5, size['height']*0.8, size['width']*0.5, size['height']*0.2, 300)
            
            time.sleep(1) # 스크롤 후 안정화

            # (3) [핵심] 로그아웃 버튼 좌표 클릭
            print(f"   👆 로그아웃 버튼 클릭")
            driver.tap([(LOGOUT_X, LOGOUT_Y)])
            
            # (4) 확인 팝업 클릭
            try:
                wait.until(EC.element_to_be_clickable((AppiumBy.ID, "android:id/button1"))).click()
                print("   ✅ 로그아웃 완료")
            except:
                print("   ⚠️ 확인 팝업이 안 뜸 (좌표가 빗나갔을 수도 있음)")

            time.sleep(2)

    except Exception as e:
        print(f"\n❌ 에러 발생: {e}")

    finally:
        driver.quit()

    # ===================== CSV + 통계 저장 =====================
        print("\n" + "=" * 50)
        print("💾 로그인 성능 결과 CSV 저장 중...")

        if results:
            # 1. 기본 데이터 프레임 생성 (회차별 데이터)
            df = pd.DataFrame({
                '회차': range(1, len(results) + 1),
                '소요시간(초)': results
            })

            # 2. 통계 계산 (평균, 최소, 최대, 표준편차)
            avg = statistics.mean(results)
            mn = min(results)
            mx = max(results)
            # 데이터가 2개 이상일 때만 표준편차 계산 (1개면 0)
            std = statistics.stdev(results) if len(results) > 1 else 0.0

            # 3. 콘솔에 요약 정보 출력
            print("\n📊 [통계 요약]")
            print(f"   평균: {avg:.4f} 초")
            print(f"   최소: {mn:.4f} 초")
            print(f"   최대: {mx:.4f} 초")
            print(f"   표준편차: {std:.4f}")

            # 4. 통계 요약 행 생성 (각 항목을 별도 행으로 추가)
            summary_rows = [
                {'회차': '평균', '소요시간(초)': f"{avg:.4f}"},
                {'회차': '최소', '소요시간(초)': f"{mn:.4f}"},
                {'회차': '최대', '소요시간(초)': f"{mx:.4f}"},
                {'회차': '표준편차', '소요시간(초)': f"{std:.4f}"}
            ]
            
            # 5. 원본 데이터 + 요약 행 합치기
            final_df = pd.concat([df, pd.DataFrame(summary_rows)], ignore_index=True)

            # 6. CSV 파일 저장
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            file_name = f"jobkorea_login_result_{REPEAT_COUNT}runs_{timestamp}.csv"
            
            final_df.to_csv(file_name, index=False, encoding='utf-8-sig')

            print(f"\n✅ CSV 저장 완료! 파일명: {file_name}")
            print(final_df) # 결과 미리보기 출력

        else:
            print("ℹ️ 저장할 측정 결과가 없어 CSV를 생성하지 않습니다.")

        print("\n✅ 모든 테스트가 완료되었습니다.")

In [4]:
# 테스트 실행
if __name__ == "__main__":
    run_jobkorea_test()

--- 잡코리아 로그인 성능 측정 ---

🔄 [ 1 / 10 ] 회차 수행 중...
📲 [1] MY 메뉴 진입
   ✅ MY 버튼 클릭 완료
   ✅ '다른 아이디로 로그인' 클릭
⌨️ [2] 정보 입력
⏱️ [3] 로그인 시작
   🎉 로그인 성공! (2.5840초)
🚪 [4] 로그아웃 진행 (좌표 클릭)
   👆 팝업 닫기
   📜 화면 최하단으로 스크롤 4회...
   👆 로그아웃 버튼 클릭
   ✅ 로그아웃 완료

🔄 [ 2 / 10 ] 회차 수행 중...
📲 [1] MY 메뉴 진입
   ✅ MY 버튼 클릭 완료
   ✅ '다른 아이디로 로그인' 클릭
⌨️ [2] 정보 입력
⏱️ [3] 로그인 시작
   🎉 로그인 성공! (2.7609초)
🚪 [4] 로그아웃 진행 (좌표 클릭)
   👆 팝업 닫기
   📜 화면 최하단으로 스크롤 4회...
   👆 로그아웃 버튼 클릭
   ✅ 로그아웃 완료

🔄 [ 3 / 10 ] 회차 수행 중...
📲 [1] MY 메뉴 진입
   ✅ MY 버튼 클릭 완료
   ✅ '다른 아이디로 로그인' 클릭
⌨️ [2] 정보 입력
⏱️ [3] 로그인 시작
   🎉 로그인 성공! (3.7071초)
🚪 [4] 로그아웃 진행 (좌표 클릭)
   👆 팝업 닫기
   📜 화면 최하단으로 스크롤 4회...
   👆 로그아웃 버튼 클릭
   ✅ 로그아웃 완료

🔄 [ 4 / 10 ] 회차 수행 중...
📲 [1] MY 메뉴 진입
   ✅ MY 버튼 클릭 완료
   ✅ '다른 아이디로 로그인' 클릭
⌨️ [2] 정보 입력
⏱️ [3] 로그인 시작
   🎉 로그인 성공! (2.5337초)
🚪 [4] 로그아웃 진행 (좌표 클릭)
   👆 팝업 닫기
   📜 화면 최하단으로 스크롤 4회...
   👆 로그아웃 버튼 클릭
   ✅ 로그아웃 완료

🔄 [ 5 / 10 ] 회차 수행 중...
📲 [1] MY 메뉴 진입
   ✅ MY 버튼 클릭 완료
   ✅ '다른 아이디로 로그인' 클릭
⌨️ [2] 정보 입력
⏱️ [3] 로그인 시작
   🎉 로그인 성공